In [1]:
import numpy as np
import math
import nnfs
from nnfs.datasets import spiral_data

nnfs.init()

In [2]:
# Create dataset
X,y =  spiral_data(samples=100, classes=3)

In [3]:
# Dense layer
class Layer_Dense:

    # Layer initialization
    def __init__(self,n_inputs, n_neurons):
        self.weights = 0.01 * np.random.rand(n_inputs, n_neurons)
        self.biases = np.zeros((1,n_neurons))

    # Forward pass
    def forward(self, inputs):
        # Calculate output values from inputs, weights and biases
        self.output = np.dot(inputs, self.weights) + self.biases

In [4]:
# ReLU Activate
class Activation_ReLU():

    # forward pass
    def forward(self, inputs):
        # Calculate output values from inputs
        self.output = np.maximum(0, inputs)


In [5]:
# Softmax activation
class Activation_Softmax():
    
    # forward pass
    def forward(self, inputs):

        # Get unnormalized probabilities
        exp_values =  np.exp(inputs - np.max(inputs,axis=1,keepdims=1))

        # Normalized them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=1)

        self.output = probabilities

In [6]:
# Common loss class

class Loss:

    # Calculates the data and regularization losses
    # given model output and ground truth values
    def calculate(self,output, y):

        # Calculate sample losses
        sample_losses = self.forward(output,y)

        data_loss = np.mean(sample_losses)

        return data_loss

In [7]:
# Cross-entropy loss

class Loss_CategoricalCrossentropy(Loss):

    # Forward pass
    def forward(self, y_pred, y_true):
        # Number of samples in a batch
        samples = len(y_pred)
        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Probabilities for target values -
        # only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples),y_true]
        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true,axis=1)
        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods


## Run inference

In [8]:
# Create Dense layer with 2 input features and 3 output values
dense1 = Layer_Dense(2, 3)

In [9]:
# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()

In [10]:
# Create second Dense layer with 3 input features (as we take output
# of previous layer here) and 3 output values
dense2 = Layer_Dense(3, 3)

In [11]:
# Create Softmax activation (to be used with Dense layer):
activation2 = Activation_Softmax()

In [12]:
# Create loss function
loss_function = Loss_CategoricalCrossentropy()

In [13]:
# Perform a forward pass of our training data through this layer
dense1.forward(X)

In [14]:

# Perform a forward pass through activation function
# it takes the output of first dense layer here
activation1.forward(dense1.output)

In [15]:
# Perform a forward pass through second Dense layer
# it takes outputs of activation function of first layer as inputs
dense2.forward(activation1.output)

In [16]:
# Perform a forward pass through activation function
# it takes the output of second dense layer here
activation2.forward(dense2.output)

In [17]:
# Let's see output of the first few samples:
print(activation2.output[:5])

[[0.33333334 0.33333334 0.33333334]
 [0.33333325 0.33333355 0.3333332 ]
 [0.33333316 0.33333376 0.33333313]
 [0.3333331  0.3333337  0.3333332 ]
 [0.33333302 0.33333385 0.3333331 ]]


In [18]:
# Perform a forward pass through loss function
# it takes the output of second dense layer here and returns loss
loss = loss_function.calculate(activation2.output, y)

In [19]:
# Print loss value
print('loss:', loss)

loss: 1.0986115


In [20]:
activation2.output

array([[0.33333334, 0.33333334, 0.33333334],
       [0.33333325, 0.33333355, 0.3333332 ],
       [0.33333316, 0.33333376, 0.33333313],
       [0.3333331 , 0.3333337 , 0.3333332 ],
       [0.33333302, 0.33333385, 0.3333331 ],
       [0.33333302, 0.3333343 , 0.33333275],
       [0.3333328 , 0.33333436, 0.33333284],
       [0.3333327 , 0.3333348 , 0.33333254],
       [0.3333326 , 0.33333495, 0.33333248],
       [0.33333254, 0.33333483, 0.33333263],
       [0.33333242, 0.33333504, 0.33333254],
       [0.33333278, 0.3333341 , 0.33333316],
       [0.33333254, 0.33333442, 0.33333302],
       [0.33333233, 0.33333504, 0.3333327 ],
       [0.33333248, 0.3333345 , 0.33333308],
       [0.33333242, 0.3333345 , 0.33333305],
       [0.3333331 , 0.3333336 , 0.33333325],
       [0.33333224, 0.3333348 , 0.33333296],
       [0.33333257, 0.33333433, 0.3333331 ],
       [0.33333197, 0.3333354 , 0.33333263],
       [0.33333144, 0.33333737, 0.33333117],
       [0.3333332 , 0.33333355, 0.33333328],
       [0.

In [21]:
# Calculate accuray from output activation2 and target
# Calculate values along first axis
predictions = np.argmax(activation2.output, axis=1)

In [22]:
if len(y.shape) == 2:
    y = np.argmax(y,axis=1)

accuracy = np.mean(predictions == y)

accuracy

np.float64(0.36)